## TF IDF outline 

- list of articles bodies 
- feed list into function to make a sparse matrix for training
- use that with tf-idf stuff above to create a tf-idf transformer 
- use that to label every individual article with tf-idf vectors in a database table


In [1]:
from sklearn.feature_extraction.text import CountVectorizer

# might take a while
bow_transformer = CountVectorizer(analyzer=text_process).fit(messages['message'])

# gets shape of input (in this case list of messages) we will use list of article bodies 
messages_bow = bow_transformer.transform(messages['message'])

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow)
tfidf4 = tfidf_transformer.transform(bow4)
print(tfidf4)

NameError: name 'text_process' is not defined